In [3]:
# === 1. 기본 설정 ===
import os
import pandas as pd
from datetime import datetime
from dotenv import load_dotenv
import bs4
import torch
import time
import json
import hashlib
from pathlib import Path

from langchain import hub
from langchain_core.documents import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI
from langchain_community.embeddings import HuggingFaceBgeEmbeddings

# 프로젝트 이름 및 환경변수 불러오기
os.environ["LANGCHAIN_PROJECT"] = "RAG_TUTORIAL"
load_dotenv()

# === 캐시 매니저 클래스 ===
class VectorStoreCache:
    """벡터스토어 캐시 관리"""
    
    def __init__(self, cache_dir="vectorstore_cache"):
        self.cache_dir = Path.home() / cache_dir
        self.cache_dir.mkdir(exist_ok=True)
        self.metadata_file = self.cache_dir / "cache_metadata.json"
        
    def _generate_cache_key(self, documents, model_name):
        """캐시 키 생성"""
        doc_sample = ""
        for doc in documents[:5]:
            doc_sample += doc.page_content[:50]
        
        config_str = f"{model_name}_{len(documents)}"
        full_content = doc_sample + config_str
        
        cache_key = hashlib.sha256(full_content.encode()).hexdigest()[:12]
        return f"rag_{cache_key}"
    
    def _load_metadata(self):
        """메타데이터 로드"""
        if self.metadata_file.exists():
            with open(self.metadata_file, 'r', encoding='utf-8') as f:
                return json.load(f)
        return {}
    
    def _save_metadata(self, metadata):
        """메타데이터 저장"""
        with open(self.metadata_file, 'w', encoding='utf-8') as f:
            json.dump(metadata, f, ensure_ascii=False, indent=2)
    
    def load_cached_vectorstore(self, documents, model_name, embedding_model):
        """캐시된 벡터스토어 로드 시도"""
        
        cache_key = self._generate_cache_key(documents, model_name)
        cache_path = self.cache_dir / cache_key
        
        print(f"🔍 캐시 확인 중... ({cache_key})")
        
        if cache_path.exists():
            try:
                print("📁 캐시 발견! 로드 중...")
                start_time = time.time()
                
                vectorstore = FAISS.load_local(str(cache_path), embedding_model, allow_dangerous_deserialization=True)
                
                load_time = time.time() - start_time
                print(f"✅ 캐시 로드 완료! 소요시간: {load_time:.2f}초")
                
                return vectorstore
                
            except Exception as e:
                print(f"⚠️ 캐시 로드 실패: {e}")
                print("새로 생성합니다...")
        else:
            print("캐시가 없습니다. 새로 생성합니다...")
        
        return None
    
    def save_vectorstore(self, vectorstore, documents, model_name):
        """벡터스토어 캐시 저장"""
        
        cache_key = self._generate_cache_key(documents, model_name)
        cache_path = self.cache_dir / cache_key
        
        print(f"💾 벡터스토어 캐시 저장 중... ({cache_key})")
        
        try:
            vectorstore.save_local(str(cache_path))
            
            # 메타데이터 저장
            metadata = self._load_metadata()
            metadata[cache_key] = {
                'created_at': datetime.now().isoformat(),
                'model_name': model_name,
                'document_count': len(documents)
            }
            self._save_metadata(metadata)
            
            print("✅ 캐시 저장 완료!")
            
        except Exception as e:
            print(f"❌ 캐시 저장 실패: {e}")
    
    def show_cache_info(self):
        """캐시 정보 출력"""
        metadata = self._load_metadata()
        
        print("\n📁 벡터스토어 캐시 정보:")
        print("-" * 40)
        
        if not metadata:
            print("캐시된 벡터스토어가 없습니다.")
            return
        
        for cache_key, info in metadata.items():
            print(f"🔑 키: {cache_key}")
            print(f"📅 생성: {info['created_at'][:16]}")
            print(f"🤖 모델: {info['model_name']}")
            print(f"📄 문서 수: {info['document_count']}")
            
            cache_path = self.cache_dir / cache_key
            if cache_path.exists():
                size_mb = sum(f.stat().st_size for f in cache_path.rglob('*')) / (1024*1024)
                print(f"💾 크기: {size_mb:.1f}MB")
            print("-" * 20)

# === 캐시를 활용한 벡터스토어 생성 함수 ===
def get_or_create_vectorstore(documents, model_name="jhgan/ko-sroberta-multitask"):
    """캐시된 벡터스토어를 로드하거나 새로 생성"""
    
    # 캐시 매니저 초기화
    cache_manager = VectorStoreCache()
    
    # 임베딩 모델 생성
    embedding = HuggingFaceBgeEmbeddings(model_name=model_name)
    
    # 캐시된 벡터스토어 로드 시도
    vectorstore = cache_manager.load_cached_vectorstore(documents, model_name, embedding)
    
    if vectorstore is not None:
        # 캐시에서 성공적으로 로드됨
        return vectorstore, cache_manager
    
    # 캐시가 없으면 새로 생성
    print("🔨 새 벡터스토어 생성 중...")
    start_time = time.time()
    
    vectorstore = FAISS.from_documents(documents, embedding=embedding)
    
    creation_time = time.time() - start_time
    print(f"✅ 벡터스토어 생성 완료! 소요시간: {creation_time:.2f}초")
    
    # 캐시에 저장
    cache_manager.save_vectorstore(vectorstore, documents, model_name)
    
    return vectorstore, cache_manager

# === 2. 웹 문서 로드 ===
url = "https://hsel.hansung.ac.kr/intro_data.mir"
web_loader = WebBaseLoader(
    web_path=(url,),
    bs_kwargs={"parse_only": bs4.SoupStrainer("div", attrs={"id": "intro_rule"})}
)
web_docs = web_loader.load()

# === 3. CSV 로드 및 월별 청크 구성 ===
# 데이터 전처리: 컴퓨터공학과 10년치 학생 데이터 학생컬럼과 소속 컬럼은 중복되는 컬럼이기에 제외시킴
# 1. 엑셀 파일 읽기
df = pd.read_excel("BookLoan_10years_data.xlsx")  # 예: "book_data.xlsx"
# 2. CSV로 저장
df.to_csv("BookLoan_10years_data.csv", index=False, encoding="utf-8-sig")

df = pd.read_csv("BookLoan_10years_data.csv", encoding="utf-8")
df["대출월"] = pd.to_datetime(df["대출일자"]).dt.to_period("M").astype(str)

csv_docs = []
for month, group in df.groupby("대출월"):
    rows = group.apply(
        lambda row: (
            f"[대출일자] {row['대출일자']} | "
            f"[학번] {row['학번']} | "
            f"[연장횟수] {row['연장횟수']} | "
            f"[청구기호] {row['청구기호']} | "
            f"[등록번호] {row['등록번호']} | "
            f"[서명] {row['서명']} | "
            f"[저자] {row['저자']} | "
            f"[보존서가] {row['보존서가 소장처']} - {row['보존서가 칸']}"
        ),
        axis=1
    )
    content = "\n".join(rows)
    csv_docs.append(Document(page_content=content, metadata={"month": month}))

# === 4. 문서 통합 및 청크 분할 ===
all_docs = web_docs + csv_docs

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1800,
    chunk_overlap=100,
    length_function=len,
)
splits = text_splitter.split_documents(all_docs)

print(f"📄 총 문서 수: {len(all_docs)}")
print(f"📄 청크 수: {len(splits)}")

# === 5. 캐시를 활용한 벡터스토어 생성 ===
print("\n🚀 벡터스토어 준비 중...")
vectorstore, cache_manager = get_or_create_vectorstore(splits)

# 캐시 정보 출력
cache_manager.show_cache_info()

# === 6. 현재 월 기준 필터링 리트리버 ===
current_month = datetime.now().strftime("%m")

def filter_by_month(docs):
    return [doc for doc in docs if doc.metadata.get("month") == current_month]

retriever = vectorstore.as_retriever(search_type="mmr", search_kwargs={"k": 10})
filtered_retriever = retriever | RunnableLambda(filter_by_month)

# === 7. LLM + 프롬프트 + 체인 구성 ===
prompt = hub.pull("rlm/rag-prompt")


llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

# === 8. 질문 실행 함수 ===
def ask(question: str):
    print("===" * 20)
    print(f"[HUMAN]\n{question}\n")
    start_time = time.time()
    response = rag_chain.invoke(question)
    response_time = time.time() - start_time
    print(f"[AI]\n{response}")
    print(f"\n⏱️ 응답 시간: {response_time:.2f}초")

# === 9. 테스트 ===
print("\n🎯 RAG 시스템 준비 완료!")
print("💡 다음 실행시에는 캐시된 벡터스토어가 즉시 로드됩니다.")
print("\n" + "="*60)

ask("책 추천해줘")

📄 총 문서 수: 133
📄 청크 수: 2004

🚀 벡터스토어 준비 중...


/var/folders/w0/ll3gcfv51rj1zzw056sn6_d00000gn/T/ipykernel_46648/2776256404.py:144: LangChainDeprecationWarning: The class `HuggingFaceBgeEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding = HuggingFaceBgeEmbeddings(model_name=model_name)


🔍 캐시 확인 중... (rag_ad024fc9efc5)
📁 캐시 발견! 로드 중...
✅ 캐시 로드 완료! 소요시간: 0.67초

📁 벡터스토어 캐시 정보:
----------------------------------------
🔑 키: rag_829742be6a2d
📅 생성: 2025-08-07T15:46
🤖 모델: jhgan/ko-sroberta-multitask
📄 문서 수: 3259
💾 크기: 18.8MB
--------------------
🔑 키: rag_ad024fc9efc5
📅 생성: 2025-08-07T18:25
🤖 모델: jhgan/ko-sroberta-multitask
📄 문서 수: 2004
💾 크기: 11.1MB
--------------------


/Users/songseung-yun/Desktop/HSU_Library_Backend/RAG/venv/lib/python3.13/site-packages/langsmith/client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(



🎯 RAG 시스템 준비 완료!
💡 다음 실행시에는 캐시된 벡터스토어가 즉시 로드됩니다.

[HUMAN]
책 추천해줘

[AI]
책 추천해드릴게요. '서민 독서 :책은 왜 읽어야 하는가'와 'Absolute Java'를 추천합니다.

⏱️ 응답 시간: 2.42초


In [4]:
# 사용자의 질문이 구체적일수록 더욱 명확한 답변제공 (성능 여부는 사용자의 책임에 있음)
ask("나는 한성대학교 3학년 재학중인 컴퓨터공학과 학생이야. 현재 금융권 IT 개발직군 취업준비중이야. 책을 추천해줘")

[HUMAN]
나는 한성대학교 3학년 재학중인 컴퓨터공학과 학생이야. 현재 금융권 IT 개발직군 취업준비중이야. 책을 추천해줘

[AI]
'이것이 취업을 위한 코딩 테스트다 with 파이썬 :취업과 이직을 결정하는 알고리즘 인터뷰 완벽 가이드' 책을 추천해드립니다. 해당 책은 IT 개발 직군 취업 준비에 도움이 될 수 있습니다. 파이썬을 활용한 코딩 테스트와 알고리즘 인터뷰에 대한 내용이 포함되어 있습니다.

⏱️ 응답 시간: 2.40초
